In [ ]:
pip install aiohttp

In [4]:
import requests
import time

import concurrent.futures
import threading

import asyncio
import aiohttp
import nest_asyncio

import multiprocessing

In [29]:
#Synchronicznie
def download_all_sites(sites):
    with requests.Session() as session:
        for url in sites:
            session.get(url)


sites = [
         
    "https://www.jython.org",
    "http://olympus.realpython.org/dice",
    
] * 80
start_time = time.time()
download_all_sites(sites)
duration_sync = time.time() - start_time

print(f"Downloaded {len(sites)} in {duration_sync} seconds")

Downloaded 160 in 6.798910140991211 seconds


In [30]:
#Przykłąd z wątkami

thread_local = threading.local()

def get_session():
    if not hasattr(thread_local, "session"):
        thread_local.session = requests.Session()
    return thread_local.session


def download_site(url):
    session = get_session()
    with session.get(url) as response:
        session.get(url)


def download_all_sites(sites):
    with concurrent.futures.ThreadPoolExecutor(max_workers=5) as executor:
        executor.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration_thread = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration_thread} seconds")

Downloaded 160 in 2.7790029048919678 seconds


In [34]:
#Asyncio - async and await
async def download_all_sites(sites):
    async with aiohttp.ClientSession() as session:
        tasks = []
        for url in sites:
            task = asyncio.ensure_future(session.get(url))
            tasks.append(task)
        await asyncio.gather(*tasks, return_exceptions=True)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    nest_asyncio.apply()
    asyncio.run(download_all_sites(sites))
    duration_asyncio = time.time() - start_time
    print(f"Downloaded {len(sites)} sites in {duration_asyncio} seconds")

Downloaded 160 sites in 9.270736932754517 seconds


In [32]:
#Multiprocessing

session = None


def set_global_session():
    global session
    if not session:
        session = requests.Session()


def download_site(url):
    with session.get(url) as response:
        name = multiprocessing.current_process().name
        session.get(url)


def download_all_sites(sites):
    with multiprocessing.Pool(initializer=set_global_session) as pool:
        pool.map(download_site, sites)


if __name__ == "__main__":
    sites = [
        "https://www.jython.org",
        "http://olympus.realpython.org/dice",
    ] * 80
    start_time = time.time()
    download_all_sites(sites)
    duration_multiproc = time.time() - start_time
    print(f"Downloaded {len(sites)} in {duration_multiproc} seconds")

Downloaded 160 in 6.883204221725464 seconds


In [37]:
#Compare
print(f"Synchronic: {duration_sync} seconds")
print(f"Threading: {duration_thread} seconds")
print(f"Asyncio: {duration_asyncio} seconds")
print(f"Multiprocessing: {duration_multiproc} seconds")

Synchronic: 6.798910140991211 seconds
Threading: 2.7790029048919678 seconds
Asyncio: 9.270736932754517 seconds
Multiprocessing: 6.883204221725464 seconds


In [38]:
#CPU-Bound Synchronous

def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    for number in numbers:
        cpu_bound(number)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration_cpu_synchronous = time.time() - start_time
    print(f"Duration {duration_cpu_synchronous} seconds")

Duration 9.37926197052002 seconds


In [40]:
#CPU-Bound multiprocessing
def cpu_bound(number):
    return sum(i * i for i in range(number))


def find_sums(numbers):
    with multiprocessing.Pool() as pool:
        pool.map(cpu_bound, numbers)


if __name__ == "__main__":
    numbers = [5_000_000 + x for x in range(20)]

    start_time = time.time()
    find_sums(numbers)
    duration_cpu_multiprocessing = time.time() - start_time
    print(f"Duration {duration_cpu_multiprocessing} seconds")

Duration 10.48751974105835 seconds


In [42]:
#
print(f"Synchronic: {duration_sync} seconds")
print(f"Threading: {duration_thread} seconds")
print(f"Asyncio: {duration_asyncio} seconds")
print(f"Multiprocessing: {duration_multiproc} seconds \n")

print(f"CPU-Bound Synchronous {duration_cpu_synchronous} seconds")
print(f"CPU-Bound multiprocessing {duration_cpu_multiprocessing} seconds")

Synchronic: 6.798910140991211 seconds
Threading: 2.7790029048919678 seconds
Asyncio: 9.270736932754517 seconds
Multiprocessing: 6.883204221725464 seconds 

CPU-Bound Synchronous 9.37926197052002 seconds
CPU-Bound multiprocessing 10.48751974105835 seconds
